 # facebook

In [3]:
#jupyter中测试运行时间  %%time
%%time
# [i for i in range(1000)]

SyntaxError: invalid syntax (<ipython-input-3-9903abfe47e0>, line 2)

In [42]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.externals import joblib

## 加载数据

In [7]:
facebook = pd.read_csv("/Users/lujiawen/Desktop/数据分析课-note/数据集/FBlocation/train.csv")
# /Volumes/TOSHIBA\ EXT/机器学习算法--hm/上头佼姐-课/7月28和7月29两天的资料/7.29课程/01-上课资料/FBlocation
facebook.shape #(29118021, 6)
facebook.head()

,row_id,x,y,accuracy,time,place_id
0,0,0.7941,9.0809,54,470702,8523065625
1,1,5.9567,4.7968,13,186555,1757726713
2,2,8.3078,7.0407,74,322648,1137537235
3,3,7.3665,2.5165,65,704587,6567393236
4,4,4.0961,1.1307,31,472130,7440663949


## 数据预处理

In [6]:
# 2.1 缩小数据范围 #这个只有我们上课才这样做。
facebook_data = facebook.query("x>2.0 & x<2.5 & y>2.0 & y<2.5")
facebook_data.shape  #(71664, 6)

/anaconda3/lib/python3.6/site-packages/numexpr/cpuinfo.py:76: UserWarning: [Errno 2] No such file or directory: 'sysctl': 'sysctl'
  stacklevel=stacklevel + 1):


(71664, 6)

#### 2 选取有用的时间特征

In [2]:
# 2.2 选择时间特征
time = pd.to_datetime(facebook_data["time"], unit="s")
time = pd.DatetimeIndex(time)  #DatetimeIndex 看看是啥 ，是不是把时间格式转换一下？？
time
facebook_data["day"] = time.day
facebook_data["hour"] = time.hour
facebook_data["weekday"] = time.weekday
facebook_data.head()

NameError: name 'pd' is not defined

#### 3 将少于n个用户的位置删除

In [1]:
# 2.3 去掉签到较少的地方
place_count = facebook_data.groupby("place_id").count()
place_count = place_count[place_count["row_id"]>3]
#打印place_count 看看
place_count
# place_count.shape #(929, 8)

# #建议赋值给一个新变量。why？？数据覆盖？？ 万一错了还有机会回头
# facebook_data_new = facebook_data[facebook_data["place_id"].isin(place_count.index)] ##?????
# facebook_data_new.shape #(69264, 9)
# facebook_data_new.head()

NameError: name 'facebook_data' is not defined

## 特征-分割数据

In [52]:
# 2.4 确定特征值和目标值
x = facebook_data_new[["x", "y", "accuracy", "day", "hour", "weekday"]] #特征
y = facebook_data_new["place_id"] #目标
# 2.5 分割数据集
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=22)
x_train.shape  
x_test.shape  ##(51948, 6)  (17316, 6)

(17316, 6)

## 特征工程-标准化

In [30]:
# 3.1 实例化一个转换器
transfer = StandardScaler()
# 3.2 调用fit_transform
x_train = transfer.fit_transform(x_train)
x_test = transfer.transform(x_test)

## 训练

In [31]:
%%time
# 4.1 实例化一个估计器
estimator = KNeighborsClassifier()
# 4.2 调用gridsearchCV
param_grid = {"n_neighbors": [1, 3, 5, 7, 9]}
estimator = GridSearchCV(estimator, param_grid=param_grid, cv=5)
# 4.3 模型训练
estimator.fit(x_train, y_train) #x:特征 y:目标

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


CPU times: user 1min 2s, sys: 826 ms, total: 1min 3s
Wall time: 1min 4s


## 预测

In [32]:
y_predict = estimator.predict(x_test)
print("最后的预测值为:\n", y_predict)

最后的预测值为:
 [9983648790 6329243787 9674001925 ... 2990018952 4830766946 7065571836]


## 评估

In [36]:
# 5.1 基本评估方式
score = estimator.score(x_test, y_test)
print("最后预测的准确率为:\n", score)
# print("预测值和真实值的对比情况:\n", y_predict == y_test)
print(sum( y_predict == y_test)/len(y_predict))

最后预测的准确率为:
 0.36515361515361516
0.36515361515361516


In [37]:
# 5.2 使用交叉验证后的评估方式
print("在交叉验证中验证的最好结果:\n", estimator.best_score_)
print("最好的参数模型:\n", estimator.best_estimator_)
print("每次交叉验证后的验证集准确率结果和训练集准确率结果:\n",estimator.cv_results_)

在交叉验证中验证的最好结果:
 0.35462385462385465
最好的参数模型:
 KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=1, p=2,
           weights='uniform')
每次交叉验证后的验证集准确率结果和训练集准确率结果:
 {'mean_fit_time': array([0.06876025, 0.06801252, 0.06029029, 0.06253452, 0.05889335]), 'std_fit_time': array([0.01174866, 0.01463424, 0.00290684, 0.00371409, 0.0019377 ]), 'mean_score_time': array([0.23749218, 0.48940473, 0.51673989, 0.6071816 , 0.6778975 ]), 'std_score_time': array([0.04202111, 0.09729483, 0.01998819, 0.04778613, 0.03480236]), 'param_n_neighbors': masked_array(data=[1, 3, 5, 7, 9],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'n_neighbors': 1}, {'n_neighbors': 3}, {'n_neighbors': 5}, {'n_neighbors': 7}, {'n_neighbors': 9}], 'split0_test_score': array([0.35231714, 0.33686953, 0.34505863, 0.34403499, 0.34105714]), 'split1_test_score': array([0.34984432, 0.34022078, 

### 模型保存

In [47]:
joblib.dump(estimator,'/Users/lujiawen/Desktop/fb_estimator.pkl')

['/Users/lujiawen/Desktop/fb_estimator.pkl']

### 模型加载

In [ ]:
 estimator = joblib.load()